In [4]:
import os
import re
from pathlib import Path
import pandas as pd
from collections import defaultdict

# ── 설정 ─────────────────────────────────────────────────────────
ROOT = Path(".")                 # 탐색 시작 경로
LOG_NAME = "99_progress.log"     # 로그 파일 이름
SAVE_WIDE = "mof_times_wide.csv" # 넓은 형태 출력 (chunk 통합 열)
SAVE_LONG = "mof_times_long.csv" # 길게(tidy) 형태 출력 (chunk 통합)
SAVE_CONFLICTS = "mof_time_conflicts.csv"  # 중복 MOF 타임 충돌 기록

# ── 유틸: 자연스러운 정렬(숫자 포함 폴더명 정렬용) ────────────────
def _natural_key(s: str):
    return [int(tok) if tok.isdigit() else tok for tok in re.split(r'(\d+)', s)]

# ── chunk 제거한 아이덴티티 ───────────────────────────────────────
#  ex) "Ar_313_15_chunk_2" -> "Ar_313_15"
CHUNK_RE = re.compile(r"(.*?)(_chunk_\d+)$")
def base_identity(folder_name: str) -> str:
    m = CHUNK_RE.match(folder_name)
    return m.group(1) if m else folder_name

# ── 라인 파서: 한 줄에서 (mof, time) 뽑기 ─────────────────────────
def parse_line(line: str):
    if "TimeForThis=" not in line:
        return None, None
    m_time = re.search(r"TimeForThis=(\d+)", line)
    if not m_time:
        return None, None
    time_val = int(m_time.group(1))

    part = line
    if "] " in line:
        part = line.split("] ", 1)[1]
    if " Done." in part:
        part = part.split(" Done.", 1)[0]
    part = part.strip()

    toks = part.split("_")
    if len(toks) >= 5:
        mof = "_".join(toks[:-4])
    else:
        mof = part

    mof = mof.strip()
    if not mof:
        return None, None
    return mof, time_val

# ── 모든 폴더의 로그 수집 ─────────────────────────────────────────
log_files = list(ROOT.rglob(f"{LOG_NAME}"))

# data[mof][base_folder] = time  (chunk 통합)
data = defaultdict(dict)
conflicts = []  # 같은 (MOF, base_folder)에 상이한 time이 여러 번 기록된 경우

for log_path in log_files:
    folder_name = log_path.parent.name
    base_folder = base_identity(folder_name)
    try:
        with open(log_path, "r", encoding="utf-8", errors="ignore") as f:
            for line in f:
                mof, time_val = parse_line(line)
                if mof is None:
                    continue
                # 이미 값이 있는데 다른 값이면 충돌 기록(마지막 값으로 덮어씀)
                old = data[mof].get(base_folder, None)
                if old is not None and old != time_val:
                    conflicts.append({
                        "MOF": mof,
                        "base_folder": base_folder,
                        "prev_time": old,
                        "new_time": time_val,
                        "chunk_file": str(log_path)
                    })
                data[mof][base_folder] = time_val
    except FileNotFoundError:
        continue

# ── DataFrame 구성 (wide) ─────────────────────────────────────────
if not data:
    print("수집된 데이터가 없습니다. 로그 형식/경로를 확인하세요.")
else:
    all_folders = sorted({bf for inner in data.values() for bf in inner.keys()}, key=_natural_key)
    all_mofs = sorted(list(data.keys()), key=_natural_key)

    wide_df = pd.DataFrame(index=all_mofs, columns=all_folders, dtype="Int64")
    for mof in all_mofs:
        for bf in all_folders:
            wide_df.at[mof, bf] = data[mof].get(bf, None)

    wide_df.to_csv(SAVE_WIDE, encoding="utf-8-sig")
    print(f"[OK] 넓은 형태 저장(Chunk 통합): {SAVE_WIDE}")

    # Long/Tidy 형태
    long_rows = []
    for mof, rowdict in data.items():
        for bf, t in rowdict.items():
            long_rows.append({"MOF": mof, "folder": bf, "time": t})
    long_df = pd.DataFrame(long_rows)
    long_df = long_df.sort_values(["MOF", "folder"], key=lambda s: s.map(str).map(_natural_key))
    long_df.to_csv(SAVE_LONG, index=False, encoding="utf-8-sig")
    print(f"[OK] 길게(tidy) 형태 저장(Chunk 통합): {SAVE_LONG}")

    # 충돌 기록(있을 때만)
    if conflicts:
        conf_df = pd.DataFrame(conflicts)
        conf_df.to_csv(SAVE_CONFLICTS, index=False, encoding="utf-8-sig")
        print(f"[INFO] 동일 (MOF, folder)에서 서로 다른 time 발견 → {SAVE_CONFLICTS}에 기록.")
    else:
        print("[INFO] 충돌 없음 (같은 MOF가 동일 아이덴티티에서 복수 chunk에 상이한 time을 갖지 않음).")

    print("\n[미리보기: wide head(10)]")
    print(wide_df.head(10))


[OK] 넓은 형태 저장(Chunk 통합): mof_times_wide.csv


TypeError: unhashable type: 'list'

In [30]:
def open_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content

txt = open_file('./99_progress.log')
lines = txt.splitlines()
lines = lines[1:-1]
def time_crop(str):
    if "TimeForThis" in str:
        time = re.findall(r'TimeForThis=(\d+)', str)[0]
        mof = '_'.join(str.split("] ")[1].split(" Done.")[0].split("_")[:-4])
        return time, mof 
    return None, None 

lis = []
for line in lines:
    
    lis.append(time_crop(line))


In [31]:
lis2 = []
for i in [ x for x in lis if x[0] is not None]:
    time = i[0]
    mof = i[1]
    dic = {'mof': mof, 'time': time}
    lis2.append(dic)
df = pandas.DataFrame(lis2)
df

,mof,time
0,DURDIF_clean,103
1,DANZEZ_charged,122
2,GIQXIO_clean,125
3,ZESFUY_clean,128
4,GIFKIP_clean,142
...,...,...
8260,QOWRAV10_clean,965
8261,VIWMOF_clean,1092
8262,TIRQOB_clean,892
8263,KAVJEX_clean,1260
